# Fine Tuning a PyTorch Model on Our Fruits Dataset
This notebook will demonstrate how to use PyTorch to fine tune a pre-existing object detection model for our fruits data set.

We will begin by creating our own DataSet class that our training module will use for data batching.

Then we will load a pre-trained Faster RCNN model using the PyTorch library and modify it so that it can classify the various fruits in our data set.

Afterwards, we will write our training, evaluation, and testing loops used for training our model.
During training, we will set it up so that if training is interrupted it can be resumed from where it stopped.
We will see how to save our "best" model found during our training loop.
Once training is completed, we will use our test data set to evaluate the precision, recall, and accuracy of our model.

Finally, we will discuss next steps, including making our model work with the PyTorch lightning framework.

## Building a Custom DataSet Class for Our Fruits Data

PyTorch provides a Dataset class that we can work with for batching during model training.
We will begin by creating our own custom Dataset class that inherits from the base PyTorch Dataset class.
Once we have our custom Dataset class, we can use the PyTorch Dataloader module to index our data for training, evaluation, and testing.
The official [PyTorch Tutorial](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) contains detailed information and links to the official documentation for both the Dataset and Dataloader class.
